In [ ]:
import requests
import datetime
import os

import pandas as pd

from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.format_scratchpad import format_to_openai_functions
from dotenv import load_dotenv

from src.prompt import MSSQL_AGENT_PREFIX
from src.tools import get_brokercheck_profile_data, get_linkedin_profile_data

load_dotenv()

In [ ]:
# csv_file_path = "./Data/AdvizorPro_Person.csv"
database_file_path = "./database/test.db"

In [ ]:
llm = ChatOpenAI(temperature=0.3,)
db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
tools = toolkit.get_tools() + [get_linkedin_profile_data] #, get_brokercheck_profile_data]

In [ ]:
import panel as pn  # GUI
import param

pn.extension()

class VTsBot(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(VTsBot, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = llm.bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt =  ChatPromptTemplate.from_messages([
            ("system", MSSQL_AGENT_PREFIX),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        if "top_k" in self.prompt.input_variables:
            self.prompt = self.prompt.partial(top_k=str(10))
        if "dialect" in self.prompt.input_variables:
            self.prompt = self.prompt.partial(dialect=toolkit.dialect)
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
        
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F3F3F3'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [ ]:
cb = VTsBot(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=2000),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard
